# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import numpy as np
import pandas as pd
import pickle
import nltk

from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.metrics import classification_report, f1_score, accuracy_score, precision_score, recall_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.multioutput import MultiOutputClassifier

In [2]:
# load data from database
engine = create_engine("sqlite:///C:\\Users\\DJ\\Documents\\git\\DSND_Term2\\project_files\\09a Disaster_response_pipeline\\workspace\\data\\DisasterResponse.db")
df = pd.read_sql_table('messages', con=engine)

In [3]:
df.head(2)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0


In [4]:
X = df['message']
y = df.iloc[:, 4:]

In [5]:
y['related'].unique()
y['related'].replace(2, y['related'].mode().iloc[0], inplace=True)

In [6]:
# X = X.head(1000)
# y = y.head(1000)

In [6]:
print(type(list(y.columns)))
list(y.columns)

<class 'list'>


['related',
 'request',
 'offer',
 'aid_related',
 'medical_help',
 'medical_products',
 'search_and_rescue',
 'security',
 'military',
 'child_alone',
 'water',
 'food',
 'shelter',
 'clothing',
 'money',
 'missing_people',
 'refugees',
 'death',
 'other_aid',
 'infrastructure_related',
 'transport',
 'buildings',
 'electricity',
 'tools',
 'hospitals',
 'shops',
 'aid_centers',
 'other_infrastructure',
 'weather_related',
 'floods',
 'storm',
 'fire',
 'earthquake',
 'cold',
 'other_weather',
 'direct_report']

In [7]:
type(X.head()), type(y)

(pandas.core.series.Series, pandas.core.frame.DataFrame)

### 2. Write a tokenization function to process your text data

In [16]:
def tokenize(text):  
    
    # Extract all the urls from the provided text
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    
    # Replace url with a url placeholder string
    for url in detected_urls:
        text = text.replace(url, 'urlplaceholder' )
    
    # didn't remove punctuation, seems to cause error 'list out of range'
    # text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # normalize text
    tokens = word_tokenize(text)
    
    # didn't remove stop words. slow down the process, also decreases model performance metrics
    # kokens = [w for w in tokens if w not in stopwords.words("english")]
    
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

In [9]:
print(X[0])
tokenize(X[0])

Weather update - a cold front from Cuba that could pass over Haiti


['weather',
 'update',
 '-',
 'a',
 'cold',
 'front',
 'from',
 'cuba',
 'that',
 'could',
 'pas',
 'over',
 'haiti']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [17]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=100))),
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [12]:
X_train.shape, y_train.shape

((19662,), (19662, 36))

In [18]:
# took about 5-10 min to run
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                  

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. 

You can do this by **iterating through the columns** and calling sklearn's `classification_report` on each.

In [19]:
y_pred = pipeline.predict(X_test)

In [20]:
y_pred.shape

(6554, 36)

In [21]:
y_test

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
15894,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16971,1,0,0,1,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
5456,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,1
20649,1,0,0,1,1,1,0,0,0,0,...,1,0,1,1,1,0,1,0,0,0
16923,1,0,0,0,0,0,0,0,0,0,...,0,0,1,1,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8261,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3685,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
18639,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13514,1,0,0,0,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,0


In [22]:
type(y_pred)

numpy.ndarray

In [23]:
y_pred[:, 0]

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [24]:
print(classification_report(y_test.iloc[:, 0], y_pred[:, 0]))

              precision    recall  f1-score   support

           0       0.74      0.27      0.39      1528
           1       0.81      0.97      0.89      5026

    accuracy                           0.81      6554
   macro avg       0.78      0.62      0.64      6554
weighted avg       0.80      0.81      0.77      6554



In [25]:
# y_test is dataframe, y_pred is numpy array

for i, col in enumerate(y.columns):
    print("[{}]".format(col))
    print(classification_report(y_test.iloc[:, i], y_pred[:, i]))

[related]
              precision    recall  f1-score   support

           0       0.74      0.27      0.39      1528
           1       0.81      0.97      0.89      5026

    accuracy                           0.81      6554
   macro avg       0.78      0.62      0.64      6554
weighted avg       0.80      0.81      0.77      6554

[request]
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      5441
           1       0.89      0.44      0.59      1113

    accuracy                           0.90      6554
   macro avg       0.89      0.71      0.76      6554
weighted avg       0.90      0.90      0.88      6554

[offer]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6529
           1       0.00      0.00      0.00        25

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      1.00      0.99   

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.97      1.00      0.99      6362
           1       0.56      0.07      0.13       192

    accuracy                           0.97      6554
   macro avg       0.77      0.54      0.56      6554
weighted avg       0.96      0.97      0.96      6554

[security]
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6436
           1       1.00      0.01      0.02       118

    accuracy                           0.98      6554
   macro avg       0.99      0.50      0.50      6554
weighted avg       0.98      0.98      0.97      6554

[military]
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      6335
           1       0.70      0.06      0.12       219

    accuracy                           0.97      6554
   macro avg       0.83      0.53      0.55      6554
weighted avg       0.96      0.97      0.95      655

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6488
           1       0.00      0.00      0.00        66

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98      6554

[earthquake]
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      5948
           1       0.89      0.71      0.79       606

    accuracy                           0.97      6554
   macro avg       0.93      0.85      0.89      6554
weighted avg       0.96      0.97      0.96      6554

[cold]
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6415
           1       0.62      0.04      0.07       139

    accuracy                           0.98      6554
   macro avg       0.80      0.52      0.53      6554
weighted avg       0.97      0.98      0.97      6554


In [26]:
# Overall scores
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='micro')
precision = precision_score(y_test, y_pred, average='micro')
recall = recall_score(y_test, y_pred, average='micro')
print("Random Forest (tuned)")
print("-"*27)
print(f"Accuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1 Score: {f1}")

Random Forest (tuned)
---------------------------
Accuracy: 0.24595666768385718
Precision: 0.8260695721711315
Recall: 0.4929846330056314
F1 Score: 0.6174721300696375


### 6. Improve your model
Use **grid search** to find better parameters. 

In [31]:
#1 RandomForestClassifier

pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize, ngram_range=(1, 1), max_features=10000)),
        ('tfidf', TfidfTransformer(use_idf=True)),
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=100))),
    ])

In [32]:
list(pipeline.get_params().keys())

['memory',
 'steps',
 'verbose',
 'vect',
 'tfidf',
 'clf',
 'vect__analyzer',
 'vect__binary',
 'vect__decode_error',
 'vect__dtype',
 'vect__encoding',
 'vect__input',
 'vect__lowercase',
 'vect__max_df',
 'vect__max_features',
 'vect__min_df',
 'vect__ngram_range',
 'vect__preprocessor',
 'vect__stop_words',
 'vect__strip_accents',
 'vect__token_pattern',
 'vect__tokenizer',
 'vect__vocabulary',
 'tfidf__norm',
 'tfidf__smooth_idf',
 'tfidf__sublinear_tf',
 'tfidf__use_idf',
 'clf__estimator__bootstrap',
 'clf__estimator__class_weight',
 'clf__estimator__criterion',
 'clf__estimator__max_depth',
 'clf__estimator__max_features',
 'clf__estimator__max_leaf_nodes',
 'clf__estimator__min_impurity_decrease',
 'clf__estimator__min_impurity_split',
 'clf__estimator__min_samples_leaf',
 'clf__estimator__min_samples_split',
 'clf__estimator__min_weight_fraction_leaf',
 'clf__estimator__n_estimators',
 'clf__estimator__n_jobs',
 'clf__estimator__oob_score',
 'clf__estimator__random_state',
 '

In [33]:
parameters = {
 #       'vect__ngram_range': ((1, 1), (1, 2)),
         'vect__max_df': (0.5, 1.0),
 #       'vect__max_features': (None, 5000, 10000),
 #       'tfidf__use_idf': (True, False),
 #       'clf__estimator__n_estimators': [50, 100, 200],
         'clf__estimator__min_samples_split': [2, 3],
        }

cv = GridSearchCV(pipeline, param_grid=parameters, cv=2, n_jobs=2, verbose=2)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [34]:
model = cv.fit(X_train, y_train)

Fitting 2 folds for each of 4 candidates, totalling 8 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   8 out of   8 | elapsed: 15.5min finished


In [35]:
y_pred = model.predict(X_test)

for i, col in enumerate(y.columns):
    print("[{}]".format(col))
    print(classification_report(y_test.iloc[:, i], y_pred[:, i]))

[related]
              precision    recall  f1-score   support

           0       0.74      0.29      0.42      1528
           1       0.82      0.97      0.89      5026

    accuracy                           0.81      6554
   macro avg       0.78      0.63      0.65      6554
weighted avg       0.80      0.81      0.78      6554

[request]
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      5441
           1       0.87      0.47      0.61      1113

    accuracy                           0.90      6554
   macro avg       0.88      0.73      0.77      6554
weighted avg       0.90      0.90      0.88      6554

[offer]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6529
           1       0.00      0.00      0.00        25

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      1.00      0.99   

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.78      0.86      0.82      3825
           1       0.77      0.66      0.71      2729

    accuracy                           0.78      6554
   macro avg       0.78      0.76      0.76      6554
weighted avg       0.78      0.78      0.77      6554

[medical_help]
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      6041
           1       0.73      0.09      0.15       513

    accuracy                           0.93      6554
   macro avg       0.83      0.54      0.56      6554
weighted avg       0.91      0.93      0.90      6554

[medical_products]
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6191
           1       0.75      0.12      0.20       363

    accuracy                           0.95      6554
   macro avg       0.85      0.56      0.59      6554
weighted avg       0.94      0.95      0


[floods]
              precision    recall  f1-score   support

           0       0.95      0.99      0.97      5983
           1       0.90      0.51      0.65       571

    accuracy                           0.95      6554
   macro avg       0.93      0.75      0.81      6554
weighted avg       0.95      0.95      0.95      6554

[storm]
              precision    recall  f1-score   support

           0       0.95      0.99      0.97      5933
           1       0.80      0.53      0.64       621

    accuracy                           0.94      6554
   macro avg       0.88      0.76      0.80      6554
weighted avg       0.94      0.94      0.94      6554

[fire]
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6488
           1       1.00      0.02      0.03        66

    accuracy                           0.99      6554
   macro avg       1.00      0.51      0.51      6554
weighted avg       0.99      0.99      0.99      

In [36]:
# RandomForestClassifier
# n_estimators = 100
print(model.best_params_)

y_pred = model.predict(X_test)

# Overall scores
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='micro')
precision = precision_score(y_test, y_pred, average='micro')
recall = recall_score(y_test, y_pred, average='micro')
print("RandomForestClassifier (tuned)")
print("-"*27)
print(f"Accuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1 Score: {f1}")

{'clf__estimator__min_samples_split': 3, 'vect__max_df': 0.5}
RandomForestClassifier (tuned)
---------------------------
Accuracy: 0.26075678974671956
Precision: 0.8211169284467714
Recall: 0.5388947217714994
F1 Score: 0.650723217887397


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [37]:
#2 KNeighborsClassifier

pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize, ngram_range=(1, 2), max_df=0.75, max_features=1000)),
        ('tfidf', TfidfTransformer(use_idf=True)),
        ('clf', MultiOutputClassifier(KNeighborsClassifier(weights='uniform', metric='minkowski'))),
    ])

parameters = {
#        'vect__ngram_range': ((1, 1), (1, 2)),
#        'vect__max_df': (0.5, 0.75, 1.0),
#        'vect__max_features': (None, 1000, 5000),
#        'tfidf__use_idf': (True, False),
         'clf__estimator__n_neighbors': (3, 5, 10),
#        'clf__estimator__weights': ['uniform', 'distance'],
#        'clf__estimator__metric': ['minkowski', 'euclidean'],
        }

cv = GridSearchCV(pipeline, param_grid=parameters, cv=2, n_jobs=2, verbose=2)
model = cv.fit(X_train, y_train)
print(list(model.best_params_))

y_pred = model.predict(X_test)

# Overall scores
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='micro')
precision = precision_score(y_test, y_pred, average='micro')
recall = recall_score(y_test, y_pred, average='micro')
print("KNeighbors (tuned)")
print("-"*27)
print(f"Accuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1 Score: {f1}")

Fitting 2 folds for each of 3 candidates, totalling 6 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   6 out of   6 | elapsed: 24.5min finished


['clf__estimator__n_neighbors']
KNeighbors (tuned)
---------------------------
Accuracy: 0.2599938968568813
Precision: 0.7515827878527739
Recall: 0.33425598930991696
F1 Score: 0.46272255805503243


In [38]:
#3 DecisionTreeClassifier

pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize, ngram_range=(1, 1), max_df=0.5, max_features=None)),
        ('tfidf', TfidfTransformer(use_idf=True)),
        ('clf', MultiOutputClassifier(DecisionTreeClassifier())),
    ])

parameters = {
#        'vect__ngram_range': ((1, 1), (1, 2)),
#        'vect__max_df': (0.5, 0.75, 1.0),
#        'vect__max_features': (None, 1000, 5000),
#        'tfidf__use_idf': (True, False),
        'clf__estimator__max_leaf_nodes': (3, 10),
        'clf__estimator__min_samples_split': [2, 3],
        }

cv = GridSearchCV(pipeline, param_grid=parameters, cv=2, n_jobs=2, verbose=2)
model = cv.fit(X_train, y_train)
print(model.best_params_)

y_pred = model.predict(X_test)

# Overall scores
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='micro')
precision = precision_score(y_test, y_pred, average='micro')
recall = recall_score(y_test, y_pred, average='micro')
print("DecisionTree (tuned)")
print("-"*27)
print(f"Accuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1 Score: {f1}")

Fitting 2 folds for each of 4 candidates, totalling 8 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   8 out of   8 | elapsed:  1.6min finished


{'clf__estimator__max_leaf_nodes': 10, 'clf__estimator__min_samples_split': 2}
DecisionTree (tuned)
---------------------------
Accuracy: 0.23298748855660664
Precision: 0.762988716872212
Recall: 0.5550730170850434
F1 Score: 0.642632189623736


### Metrics comparison of three models

|Estimator|Accuracy|Precision|Recall|F1 Score|
|---|---|---|---|---|
|(1) RandomForestClassifier|0.26|0.82|0.53|0.65|
|(2) KNeighborsClassifier|0.25|0.75|0.33|0.46|
|(3) DecisionTreeClassifier|0.23|0.76|0.55|0.64|


Based on accuracy, precision, recall and F1 score, RandomForestClassifier has the best overall performance and was selected for the final model. To further improve the model, two additional features were added using FeatureUnion.

### Add two more features

In [39]:
# add 1st feature

class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']:
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [40]:
sve = StartingVerbExtractor()

In [41]:
sve.transform(X_train)

,message
9318,True
12480,False
3887,True
10310,True
4312,True
...,...
11397,False
4648,False
25899,False
8946,False


In [42]:
# add 2nd feature

class TextLenghExtractor(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = pd.Series(X).apply(lambda x: len(x))
        return pd.DataFrame(X)

In [43]:
# using FeatureUnion, add two features StartingVerbExtractor and TextLenghExtractor

pipeline = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize, ngram_range=(1,1), max_df=0.75, max_features=10000)),
            ('tfidf', TfidfTransformer(use_idf=True))
        ])),

        ('starting_verb', StartingVerbExtractor()),      
        ('text_length', TextLenghExtractor())
    ])),

    ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=100)))
])


In [44]:
# specify parameters for grid search

parameters = {
#    'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
#    'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
#    'features__text_pipeline__vect__max_features': (None, 5000, 10000),
#    'features__text_pipeline__tfidf__use_idf': (True, False),
#    'clf__estimator__n_estimators': [50, 100, 200],
    'clf__estimator__min_samples_split': [2, 3],

    'features__transformer_weights': (
        {'text_pipeline': 1, 'starting_verb': 0.5, 'text_length': 0.5},
        {'text_pipeline': 1, 'starting_verb': 0.5, 'text_length': 1},
        {'text_pipeline': 1, 'starting_verb': 1, 'text_length': 1},
    )
}

In [45]:
cv = GridSearchCV(pipeline, param_grid=parameters, cv=2, n_jobs=2, verbose=2)

model = cv.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(model.best_params_)

# Overall scores
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='micro')
precision = precision_score(y_test, y_pred, average='micro')
recall = recall_score(y_test, y_pred, average='micro')
print("RandomForestClassifier (tuned)")
print("-"*27)
print(f"Accuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1 Score: {f1}")

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  12 out of  12 | elapsed: 31.2min finished


{'clf__estimator__min_samples_split': 2, 'features__transformer_weights': {'text_pipeline': 1, 'starting_verb': 1, 'text_length': 1}}
RandomForestClassifier (tuned)
---------------------------
Accuracy: 0.2525175465364663
Precision: 0.819724839240317
Recall: 0.5231936623079125
F1 Score: 0.6387205779538568


In [46]:
model.best_params_

{'clf__estimator__min_samples_split': 2,
 'features__transformer_weights': {'text_pipeline': 1,
  'starting_verb': 1,
  'text_length': 1}}

### 9. Export your model as a pickle file

In [47]:
pickle.dump(cv, open('workspace\models\classifier.pkl', 'wb'))

In [48]:
# double check if the pickle file works
loaded_model = pickle.load(open('workspace\models\classifier.pkl', 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.2525175465364663


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.